# Run SetFit model with OpenVINO

In this notebook, we'll learn how to export SetFit model to OpenVINO and run the inference.

## Setup

If you're running this Notebook on Colab or some other cloud platform, you will need to install the `setfit` library. Uncomment the following cell and run it:

In [1]:
# %pip install setfit[openvino]

## Load pretrained SetFit model from the Hub

In [2]:
from time import perf_counter
from setfit import SetFitModel

NUM_ITERS=100

model_id = "lewtun/my-awesome-setfit-model"
model = SetFitModel.from_pretrained("lewtun/my-awesome-setfit-model")

# Run inference
input_text = ["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]

start = perf_counter()
for i in range(NUM_ITERS):
    preds = model(input_text)
end = perf_counter()

preds, f"{end-start:.2f} seconds"

/home/alex/virt_envs/setfit/lib/python3.8/site-packages/tqdm-4.64.1-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alex/virt_envs/setfit/lib/python3.8/site-packages/scikit_learn-1.1.3-py3.8-linux-x86_64.egg/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.1.1 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(array([1, 0]), '1.87 seconds')

## Export to OpenVINO

At this step, the model is exported to OpenVINO Intermediate Representation (IR). `setfit` also provides a pure PyTorch implementation of `SetFitModel`, where the head is a dense layer instead of a classifier from `scikit-learn`. This allows one to do backprop end-to-end and have more fine-grained control over the training process. The model body (Transformer) and head are merged into the one model during the export.

To use the PyTorch model, we load a pretrained model with `use_differentiable_head=True` and specify the number of classes to include in the head:

In [4]:
from setfit import SetFitModel
from setfit.exporters.openvino import export_to_openvino

output_path = "model.xml"
export_to_openvino(model, output_path)

UnboundLocalError: local variable 'onnx_path' referenced before assignment

## Run model with OpenVINO
Now, we run the model wit OpenVINO Python* API to compare results. The elapsed inference time is computed over `NUM_ITERS` to compare with PyTorch.

>**Note:** Text tokenization is a required extra step that should be done before the inference

In [ ]:
from transformers import AutoTokenizer
import openvino.runtime as ov

tokenizer = AutoTokenizer.from_pretrained(model_id)
inputs = tokenizer(
    input_text,
    padding=True,
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=True,
    return_tensors="np",
)

inputs_dict = dict(inputs)

core = ov.Core()
ov_model = core.read_model(output_path)
compiled_model = core.compile_model(ov_model, "CPU")

start = perf_counter()
for i in range(NUM_ITERS):
    ov_preds = compiled_model(inputs_dict)[compiled_model.outputs[0]]
end = perf_counter()

ov_preds, f"{end-start:.2f} seconds"


(array([1, 0]), '1.26 seconds')